In [ ]:
from pyspark.sql import SparkSession  # Import the SparkSession class

spark = SparkSession.builder \
    .appName("MyApp") \
    .getOrCreate()

print("Spark Version : " + spark.version)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/02 18:56:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark Version : 3.5.1


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 53669)
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/Users/cristianooliveira/Documents/ModelTrack/.venv/lib/python3.10/site-packages/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/Users/cristianoolivei

# Exemplo com Dataframe Pandas

In [10]:
import pandas as pd

from modeltrack.feature_engineering.binning import TreeBinner, CutBinner
from modeltrack.feature_engineering.encoder import WOEEncoder

# -----------------------------
# 1. Dados de exemplo
# -----------------------------
df_pd = pd.DataFrame({
    "idade": [22, 25, 45, 33, 40, 50, 60, 35, None],
    "renda": [1500, 1800, 7000, 2500, 3000, 10000, 8000, 1200, None],
    "sexo": ["M", "F", "M", "F", "M", "M", "F", "M", "F"],
    "target": [0, 0, 1, 0, 1, 1, 0, 1, 1]
})

# -----------------------------
# 2. Pandas: binning
# -----------------------------
# Idade
features = ["idade", "renda"]
binner = TreeBinner(max_depth=2)

for feature in features:
    binner.fit(X=df_pd[feature], y=df_pd["target"], col_name=feature)

df_pd = binner.transform_dataframe(df_pd, features=features, suffix="_binned")

# -----------------------------
# 3. Pandas: WOE encoding
# -----------------------------
# Idade binned

woe_encoder = WOEEncoder()
features = ["idade_binned", "renda_binned", "sexo"]
for feature in features:
    woe_encoder.fit(X=df_pd[feature], y=df_pd["target"], col_name=feature)

df_pd = woe_encoder.transform_dataframe(df_pd, features=features, suffix="_woe")
print("==== Pandas: Transformação completa ====")
df_pd

==== Pandas: Transformação completa ====


,idade,renda,sexo,target,idade_binned,renda_binned,idade_binned_woe,renda_binned_woe,sexo_woe
0,22.0,1500.0,M,0,<34,"[1350, 2750)",-1.540445,-1.540445,0.762140
1,25.0,1800.0,F,0,<34,"[1350, 2750)",-1.540445,-1.540445,-0.847298
2,45.0,7000.0,M,1,"[34, 55)","[2750, 7500)",1.455287,0.944462,0.762140
3,33.0,2500.0,F,0,<34,"[1350, 2750)",-1.540445,-1.540445,-0.847298
4,40.0,3000.0,M,1,"[34, 55)","[2750, 7500)",1.455287,0.944462,0.762140
5,50.0,10000.0,M,1,"[34, 55)",>=7500,1.455287,-0.154151,0.762140
6,60.0,8000.0,F,0,>=55,>=7500,-0.847298,-0.154151,-0.847298
7,35.0,1200.0,M,1,"[34, 55)",<1350,1.455287,0.538997,0.762140
8,NaN,NaN,F,1,N/A,N/A,0.538997,0.538997,-0.847298


# Exemplo com Dataframe Spark

In [3]:
# -----------------------------
# 3. Spark: binning + WOE encoding
# -----------------------------
spark = SparkSession.builder.master("local[1]").appName("FeatureEngineeringExample").getOrCreate()
df_pd = pd.DataFrame({
    "idade": [22, 25, 45, 33, 40, 50, 60, 35, None],
    "renda": [1500, 1800, 7000, 2500, 3000, 10000, 8000, 1200, None],
    "sexo": ["M", "F", "M", "F", "M", "M", "F", "M", "F"],
    "target": [0, 0, 1, 0, 1, 1, 0, 1, 1]
})

df_spark = spark.createDataFrame(df_pd)

# Binagem Spark
features = ["idade", "renda"]
binner = TreeBinner(max_depth=3)

for f in features:
    binner.fit(df_spark.select(f).toPandas()[f], df_spark.select("target").toPandas()["target"], col_name=f)

# Transform de várias colunas de uma vez
df_spark = binner.transform_dataframe(df_spark, ["idade", "renda"])

woe_encoder = WOEEncoder()
features = ["idade_binned", "renda_binned", "sexo"]
for feature in features:
    woe_encoder.fit(X=df_spark.select(feature).toPandas()[feature], y=df_spark.select("target").toPandas()["target"], col_name=feature)
df_spark = woe_encoder.transform(df_spark, columns=["idade_binned", "renda_binned", "sexo"])

print("==== Spark: Transformação completa ====")
df_spark.show(truncate=False)

25/09/02 15:52:40 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


==== Spark: Transformação completa ====
+-----+-------+----+------+------------+------------+-------------------+-------------------+-------------------+
|idade|renda  |sexo|target|idade_binned|renda_binned|idade_binned_woe   |renda_binned_woe   |sexo_woe           |
+-----+-------+----+------+------------+------------+-------------------+-------------------+-------------------+
|22.0 |1500.0 |M   |0     |<34         |[1350, 2750)|-1.540445040947149 |-1.540445040947149 |0.7621400520468969 |
|25.0 |1800.0 |F   |0     |<34         |[1350, 2750)|-1.540445040947149 |-1.540445040947149 |-0.8472978603872037|
|45.0 |7000.0 |M   |1     |[34, 55)    |[2750, 7500)|1.4552872326068422 |0.9444616088408515 |0.7621400520468969 |
|33.0 |2500.0 |F   |0     |<34         |[1350, 2750)|-1.540445040947149 |-1.540445040947149 |-0.8472978603872037|
|40.0 |3000.0 |M   |1     |[34, 55)    |[2750, 7500)|1.4552872326068422 |0.9444616088408515 |0.7621400520468969 |
|50.0 |10000.0|M   |1     |[34, 55)    |>=9000  

# Exemplo aplicando coluna a coluna

In [4]:
import pandas as pd

from modeltrack.feature_engineering.binning import TreeBinner, CutBinner
from modeltrack.feature_engineering.encoder import WOEEncoder

# -----------------------------
# 1. Dados de exemplo
# -----------------------------
df_pd = pd.DataFrame({
    "idade": [22, 25, 45, 33, 40, 50, 60, 35, None],
    "renda": [1500, 1800, 7000, 2500, 3000, 10000, 8000, 1200, None],
    "sexo": ["M", "F", "M", "F", "M", "M", "F", "M", "F"],
    "target": [0, 0, 1, 0, 1, 1, 0, 1, 1]
})

# -----------------------------
# 2. Pandas: binning
# -----------------------------
# Idade
features = ["idade", "renda"]
binner = TreeBinner(max_depth=2)

for feature in features:
    binner.fit(X=df_pd[feature], y=df_pd["target"], col_name=feature)
    df_pd = binner.transform_dataframe(df_pd, features=[feature], suffix="_binned")

# -----------------------------
# 3. Pandas: WOE encoding
# -----------------------------
# Idade binned

woe_encoder = WOEEncoder()
features = ["idade_binned", "renda_binned", "sexo"]
for feature in features:
    woe_encoder.fit(X=df_pd[feature], y=df_pd["target"], col_name=feature)
    df_pd = woe_encoder.transform_dataframe(df_pd, features=[feature], suffix="_woe")
print("==== Pandas: Transformação completa ====")
df_pd

==== Pandas: Transformação completa ====


,idade,renda,sexo,target,idade_binned,renda_binned,idade_binned_woe,renda_binned_woe,sexo_woe
0,22.0,1500.0,M,0,<34,"[1350, 2750)",-1.540445,-1.540445,0.762140
1,25.0,1800.0,F,0,<34,"[1350, 2750)",-1.540445,-1.540445,-0.847298
2,45.0,7000.0,M,1,"[34, 55)","[2750, 7500)",1.455287,0.944462,0.762140
3,33.0,2500.0,F,0,<34,"[1350, 2750)",-1.540445,-1.540445,-0.847298
4,40.0,3000.0,M,1,"[34, 55)","[2750, 7500)",1.455287,0.944462,0.762140
5,50.0,10000.0,M,1,"[34, 55)",>=7500,1.455287,-0.154151,0.762140
6,60.0,8000.0,F,0,>=55,>=7500,-0.847298,-0.154151,-0.847298
7,35.0,1200.0,M,1,"[34, 55)",<1350,1.455287,0.538997,0.762140
8,NaN,NaN,F,1,N/A,N/A,0.538997,0.538997,-0.847298


# Exemplo Feature Pipeline

In [9]:
import pandas as pd
from pyspark.sql import SparkSession
from modeltrack.feature_engineering.feature_pipeline import FeaturePipelineDev, FeaturePipelinePrd

# ============================
# 1. Criar dados de exemplo (20 linhas)
# ============================
df_pd = pd.DataFrame({
    "idade": [22, 25, 45, 33, 40, 50, 60, 35, None, 28,
              31, 42, 55, 38, None, 47, 29, 52, 61, 36],
    "renda": [1500, 1800, 7000, 2500, 3000, 10000, 8000, 1200, None, 2200,
              2700, 6500, 7200, 3100, 4000, 5600, 1950, 8800, None, 3300],
    "sexo": ["M", "F", "M", "F", "M", "M", "F", "M", "F", "M",
             "F", "M", "F", "M", "M", "F", "M", "F", "M", "F"],
    "target": [0, 0, 1, 0, 1, 1, 0, 1, 1, 0,
               1, 0, 1, 0, 1, 1, 0, 1, 0, 1]
})

spark = SparkSession.builder.master("local[1]").appName("FeaturePipelineExample").getOrCreate()
df_spark = spark.createDataFrame(df_pd)

# ============================
# 2. DEV: Fit em apenas uma parte (ex: 40%)
# ============================
df_sample = df_pd.sample(frac=0.4, random_state=42)  # só parte dos dados

pipeline_dev = FeaturePipelineDev(binning_method="tree", max_depth=2)

df_transformed = pipeline_dev.fit_transform(
    df=df_sample,                       # usa só a amostra para FIT
    features_binning=["idade", "renda"],
    features_encoding=["idade_binned", "renda_binned", "sexo"],
    target="target"
)

# salvar parâmetros treinados
pipeline_dev.save("artifacts_pipeline")

print("==== DEV (fit em sample) ====")
df_transformed


==== DEV (fit em sample) ====


,idade,renda,sexo,target,idade_binned,renda_binned,idade_binned_woe,renda_binned_woe,sexo_woe
0,22.0,1500.0,M,0,<44.5,<4050,-1.609438,-1.386294,-0.405465
17,52.0,8800.0,F,1,>=44.5,>=7650,1.386294,1.098612,0.287682
15,47.0,5600.0,F,1,>=44.5,"[4050, 7650)",1.386294,0.000000,0.287682
1,25.0,1800.0,F,0,<44.5,<4050,-1.609438,-1.386294,0.287682
8,NaN,NaN,F,1,N/A,N/A,0.693147,0.693147,0.287682
5,50.0,10000.0,M,1,>=44.5,>=7650,1.386294,1.098612,-0.405465
11,42.0,6500.0,M,0,<44.5,"[4050, 7650)",-1.609438,0.000000,-0.405465
3,33.0,2500.0,F,0,<44.5,<4050,-1.609438,-1.386294,0.287682


In [8]:
# ============================
# 3. PRD: Aplicar em todo o dataframe
# ============================
pipeline_prd = FeaturePipelinePrd("artifacts_pipeline")

df_full_transformed = pipeline_prd.transform(
    df=df_pd,  # agora em todos os dados
    features_binning=["idade", "renda"],
    features_encoding=["idade_binned", "renda_binned", "sexo"]
)

print("==== PRD (transform no dataset inteiro) ====")
df_full_transformed


==== PRD (transform no dataset inteiro) ====


,idade,renda,sexo,target,idade_binned,renda_binned,idade_binned_woe,renda_binned_woe,sexo_woe
0,22.0,1500.0,M,0,<44.5,<4050,-1.609438,-1.386294,-0.405465
1,25.0,1800.0,F,0,<44.5,<4050,-1.609438,-1.386294,0.287682
2,45.0,7000.0,M,1,>=44.5,"[4050, 7650)",1.386294,0.000000,-0.405465
3,33.0,2500.0,F,0,<44.5,<4050,-1.609438,-1.386294,0.287682
4,40.0,3000.0,M,1,<44.5,<4050,-1.609438,-1.386294,-0.405465
5,50.0,10000.0,M,1,>=44.5,>=7650,1.386294,1.098612,-0.405465
6,60.0,8000.0,F,0,>=44.5,>=7650,1.386294,1.098612,0.287682
7,35.0,1200.0,M,1,<44.5,<4050,-1.609438,-1.386294,-0.405465
8,NaN,NaN,F,1,N/A,N/A,0.693147,0.693147,0.287682
9,28.0,2200.0,M,0,<44.5,<4050,-1.609438,-1.386294,-0.405465


In [7]:
# ============================
# 4. Spark também (opcional)
# ============================
df_spark_transformed = pipeline_prd.transform(
    df=df_spark,
    features_binning=["idade", "renda"],
    features_encoding=["idade_binned", "renda_binned", "sexo"]
)

print("==== Spark: PRD ====")
df_spark_transformed.show(truncate=False)

==== Spark: PRD ====
+-----+-------+----+------+------------+------------+-------------------+-------------------+--------------------+
|idade|renda  |sexo|target|idade_binned|renda_binned|idade_binned_woe   |renda_binned_woe   |sexo_woe            |
+-----+-------+----+------+------------+------------+-------------------+-------------------+--------------------+
|22.0 |1500.0 |M   |0     |<44.5       |<4050       |-1.6094379124341005|-1.3862943611198906|-0.40546510810816444|
|25.0 |1800.0 |F   |0     |<44.5       |<4050       |-1.6094379124341005|-1.3862943611198906|0.28768207245178085 |
|45.0 |7000.0 |M   |1     |>=44.5      |[4050, 7650)|1.3862943611198906 |0.0                |-0.40546510810816444|
|33.0 |2500.0 |F   |0     |<44.5       |<4050       |-1.6094379124341005|-1.3862943611198906|0.28768207245178085 |
|40.0 |3000.0 |M   |1     |<44.5       |<4050       |-1.6094379124341005|-1.3862943611198906|-0.40546510810816444|
|50.0 |10000.0|M   |1     |>=44.5      |>=7650      |1.3862